In [ ]:
import sqlalchemy as sa
import pandas as pd

DB_URL = "postgresql+psycopg2://scout:scout@db:5432/scouting"
engine = sa.create_engine(DB_URL)

# total de jugadores
with engine.connect() as conn:
    n_players = conn.scalar(sa.text("SELECT COUNT(*) FROM players"))
print(f"Hay {n_players:,} jugadores")

In [ ]:
df_players = pd.read_sql_table("players", con=engine)

In [ ]:
df_players.head()

In [ ]:
df_players.info()

## Creating feature_vector Column

In [ ]:
from sqlalchemy import create_engine, text

# 1️⃣  Conexión -------------------------------------------------
engine = create_engine(
    "postgresql+psycopg2://scout:scout@db:5432/scouting",
    isolation_level="AUTOCOMMIT",  # evita tener que hacer COMMIT explícito
)

# 2️⃣  Parámetros que necesitas saber de antemano --------------
PLAYER_DIM = 43   # ← dimensión real de tu vector de jugador
LISTS      = 140   # ← granularidad del índice IVFFLAT

with engine.begin() as conn:
    # 3️⃣  Activa la extensión pgvector (idempotente) -----------
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))

    # 4️⃣  Añade la nueva columna vector(N) y migra los datos ---
    conn.execute(text(f"""
        ALTER TABLE players
          ADD COLUMN IF NOT EXISTS feature_tmp vector({PLAYER_DIM});

        UPDATE players
          SET feature_tmp = feature_vector::vector
          WHERE feature_tmp IS NULL;      -- evita tocar filas ya convertidas

        ALTER TABLE players
          DROP COLUMN IF EXISTS feature_vector;
        ALTER TABLE players
          RENAME COLUMN feature_tmp TO feature_vector;
    """))

    # 5️⃣  Crea (o recrea) el índice IVFFLAT --------------------
    conn.execute(text(f"""
        CREATE INDEX IF NOT EXISTS players_feature_vec_idx
        ON players USING ivfflat (feature_vector vector_cosine_ops)
        WITH (lists = {LISTS});
    """))

print("Migración a pgvector finalizada ✔️")

### Populate feature_vector Column

In [ ]:
FEATURE_COLS = [
    "minutes", "minutes_90s",
    "goals", "assists",
    "expected_goals", "expected_assists",
    "no_penalty_expected_goals_plus_expected_assists",
    "progressive_carries", "progressive_passes", "progressive_passes_received",
    "goals_per90", "assists_per90", "goals_assists_per90",
    "expected_goals_per90", "expected_assists_per90", "expected_goals_assists_per90",
    "gk_goals_against", "gk_pens_allowed", "gk_free_kick_goals_against",
    "gk_corner_kick_goals_against", "gk_own_goals_against",
    "gk_psxg", "gk_psnpxg_per_shot_on_target_against",
    "passes_completed", "passes", "passes_pct",
    "passes_progressive_distance", "passes_completed_long", "passes_long", "passes_pct_long",
    "tackles", "tackles_won", "challenge_tackles", "challenges",
    "challenge_tackles_pct", "challenges_lost",
    "blocks", "blocked_shots", "blocked_passes",
    "interceptions", "tackles_interceptions", "clearances", "errors"
]

df = pd.read_sql(
    f"SELECT id, {', '.join(FEATURE_COLS)} FROM players",
    engine
)

In [ ]:
df

#### Standarization feature_vector values

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
emb_matrix = scaler.fit_transform(df[FEATURE_COLS]).astype("float32")

In [ ]:
df["feature_vector_py"] = [vec.tolist() for vec in emb_matrix]

In [ ]:
df.info()

In [ ]:
from pgvector.sqlalchemy import Vector

with engine.begin() as conn:
    conn.execute(
        text("""
            UPDATE players
            SET feature_vector = :vec
            WHERE id = :pid
        """),
        [{"pid": pid, "vec": vec} for pid, vec in zip(df.id, df.feature_vector_py)]
    )

#### Checks

In [ ]:
check = pd.read_sql(
    "SELECT vector_dims(feature_vector) AS dim, COUNT(*) FROM players GROUP BY dim;",
    engine
)
assert check["dim"].iloc[0] == 43, "Alguna fila no tiene dimensión 43"
print(check)

In [ ]:
with engine.begin() as conn:
    conn.execute(text("ANALYZE players;"))
    conn.execute(text("SET ivfflat.probes = 10;")) 

In [ ]:
query = """
EXPLAIN (ANALYZE, BUFFERS)
SELECT * FROM players
ORDER BY feature_vector <=> (SELECT feature_vector FROM players WHERE id = 42)
LIMIT 15;
"""

In [ ]:
with engine.begin() as conn:
    conn.execute(text(query))

In [ ]:
df_plan = pd.read_sql(query, engine)
display(df_plan)

In [ ]:
df

## Football news Table validations 

In [ ]:
# primeras 5 noticias
query = """
SELECT *
FROM football_news
ORDER BY published_at DESC
LIMIT 5
"""
df = pd.read_sql(query, con=engine)
df